In [1]:
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from collections import Counter
from scipy.sparse import lil_matrix
from tqdm import tqdm

In [2]:
n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20

In [3]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [4]:
data_samples=[]
for line in open('./wechat.txt'):
    data_samples.append(line.strip())

In [6]:
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features)
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf features for LDA...
done in 61.562s.



In [7]:
tf_feature_names = tf_vectorizer.get_feature_names()#这是词表,名字一样是因为我们只需要后面这个，前面的可以覆盖了
fout4=open('./feature_names.txt','w')
fout4.write('\n'.join(tf_feature_names))
index=range(1,(len(tf_feature_names)+1))#这是下标列表
#zip的作用你可以试一下
tf_feature_names=dict(zip(tf_feature_names,index))#这里我们创建了一个dict，key是词，value是下标
fout1=open('./transfered_data_text.txt','w')
fout2=open('./transfered_data_id.txt','w')
fout3=open('./transfered_data_libsvm.txt','w')

for i,d in enumerate(data_samples):
    #所以d是一篇文章，是一个string，i是下标，从0开始，这是enumerate这个函数的作用
    #首先需要分割
    #然后我们需要过滤
    #这里的lambda的意思
    d1=list(filter(lambda w:w in tf_feature_names,d.split(' ')))
    fout1.write(' '.join(d1)+'\n')
    d2=list(map(lambda w:str(tf_feature_names[w]),d1))
    fout2.write(' '.join(d2)+'\n')
    d3=Counter(d2)
    d3=list(d3.items())
    d3.sort(key=lambda x:int(x[0]))
    d3=list(map(lambda v:'%s:%d'%(v),d3))
    fout3.write(str(i)+' '+' '.join(d3)+'\n')

In [12]:
fout4.close()

In [8]:
data_samples=[]
data = lil_matrix((13872, 1000))
for i,line in tqdm(enumerate(open('./transfered_data_libsvm.txt'))):
    line=line.strip().split(' ')
    data_samples.append([])
    for v in line[1:]:
        v=v.split(':')
        data[i,int(v[0])-1]=int(v[1])

13243it [00:29, 443.16it/s]


In [9]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(data)
print("done in %0.3fs." % (time() - t0))

Fitting LDA models with tf features, n_samples=2000 and n_features=1000...
done in 121.400s.


In [13]:
print("\nTopics in LDA model:")
feature_names = [line.strip() for line in open('./feature_names.txt')]
print(len(feature_names))
print_top_words(lda, feature_names, n_top_words)


Topics in LDA model:
1000
Topic #0: 宝宝 健康 皮肤 身体 分钟 作用 疾病 时间 食物 效果 营养 即可 人体 治疗 医生 方法 导致 功能 医院 鸡蛋
Topic #1: 活动 时间 工作 电话 地址 酒店 旅游 上海 招聘 免费 优惠 微信 报名 公司 中心 服务 客户 北京 公园 工资
Topic #2: 关注 二维码 识别 女人 免费 简介 订阅 微信 男人 生活 点击 分享 健康 选择 添加 技巧 公众 轻松 搭配 知识
Topic #3: 公司 产品 企业 技术 用户 数据 行业 市场 互联网 平台 品牌 服务 手机 发展 中国 系统 管理 员工 模式 客户
Topic #4: 微信 点击 阅读 浏览器 播放 原文 回复 音乐 手机 支持 朋友 文章 查看 视频 公众 语音 内容 关注 时间 下方
Topic #5: 公司 市场 投资 银行 汽车 金融 基金 资金 风险 上市 资产 价格 交易 机构 股票 业务 股份 行业 车型 企业
Topic #6: 设计 电影 生活 搭配 空间 喜欢 选择 品牌 世界 中国 作品 第一 风格 感觉 时间 系列 时尚 自然 特别 适合
Topic #7: 孩子 生活 女人 男人 朋友 喜欢 父母 人生 时间 妈妈 事情 工作 学习 发现 东西 老师 世界 家庭 第一 幸福
Topic #8: 中国 美国 经济 国家 城市 韩国 发展 北京 房价 政府 房地产 市场 日本 乐天 企业 世界 社会 投资 政策 第一
Topic #9: 工作 建设 管理 发展 项目 人员 服务 企业 国家 工程 单位 全国 交通 相关 信息 教育 社会 标准 部门 专业

